See how to use Shor's algorithm to factor 15 here:    <b><a href="https://portal.quantumrings.com/doc/Shors.html">Shor15</a></b>

<i><b>Source code to factorize 15</b></i>

Note: Be sure to use your API token and your account name.

Step 1. Import the required modules and obtain the backend

In [1]:
#!pip install qiskit[visualization]

In [2]:
import qiskit
print (qiskit.__version__)

1.3.0


In [3]:
#!pip install QuantumRingsLib

In [4]:
!python --version

Python 3.12.4


In [5]:
#!pip install quantumrings-toolkit-qiskit

In [6]:
#!pip install qiskit-aer

In [7]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT

from qiskit_aer.primitives import Sampler
from qiskit.visualization import plot_histogram

In [8]:
def a2jmodN(a, j, N):
    """Compute a^{2^j} (mod N) by repeated squaring"""
    for _ in range(j):
        a = np.mod(a**2, N)
    return a

def qft_dagger(n):
    """Creates an n-qubit inverse Quantum Fourier Transform circuit."""
    qc = QFT(num_qubits=n, do_swaps=False).inverse()
    return qc.to_gate(label="QFT†")

In [9]:
from fractions import Fraction
from math import gcd,ceil,log2, pi

def c_amod15_2(a,N):
    """
    Controlled multiplication by a mod N using QFT instead of SWAP.
    """
    Q = ceil(log2(N))
    U = QuantumCircuit(Q)
    for i in range(Q // 2):
        U.swap(i, Q-i-1)

    if a//2 ==1:
        for q in range(Q):      
            U.x(q)
    
    #U.append(qft_dagger(Q), range(Q))
    qft_circuit = QFT(Q).decompose()
    qft_circuit.compose(U)
    print(f"num_qubits:{qft_circuit.num_qubits}")

    #display(qft_circuit.draw("mpl"))
    qft_circuit = qft_circuit.to_gate()
    qft_circuit.name = f"{a} mod {Q}"
    c_U = qft_circuit.control()
    return c_U
                

In [52]:
def phase_estimation_2(
        controlled_operation: QuantumCircuit,
        psi_prep: QuantumCircuit,
        precision: int,
        #theta: float,
    ):
    """
    Carry out phase estimation on a simulator.
    Args:
        controlled_operation: The operation to perform phase estimation on,
                              controlled by one qubit.
        psi_prep: Circuit to prepare |ψ>
        precision: Number of counting qubits to use
    Returns:
        float: Best guess for phase of U|ψ>
    """
    control_register = QuantumRegister(precision)
    output_register = ClassicalRegister(precision)

    target_register = QuantumRegister(psi_prep.num_qubits)
    qc = QuantumCircuit(control_register, target_register, output_register)

    # Prepare |ψ>
    qc.compose(psi_prep,
               qubits=target_register,
               inplace=True)

    # Do phase estimation
    # theta = 0.7 # for 8 bits
    # theta = 0.25 # for 10 bits
    # theta = 0.1 # for 12, 14, 16, 18 bits
    # theta = 0.01 # for 18 bits
    # theta = 0.025 # for 20 bits
    theta = 0.0125 # for 22 bits
    
    for index, qubit in enumerate(control_register):
        qc.h(qubit)
        qc.cp(2 * pi * theta*(index+1), qubit, target_register)
    qc.barrier()

    qc.compose(
        QFT(precision, inverse=True),
        qubits=control_register,
        inplace=True
    )

    # Count the operations
    gate_counts = qc.size()
    print(f"gate_counts:{gate_counts}")

    qc.measure(control_register, output_register)

    measurement = Sampler().run(qc, shots=1).result().quasi_dists[0].popitem()[0]
    return measurement / 2**precision

In [53]:
import semiprimes
data = semiprimes.semiprimes
items = list(data.items())
a = items[7][0]
N = items[7][1]

Q = ceil(log2(N))
psi_prep = QuantumCircuit(4)
psi_prep.x(0)
#display(psi_prep.draw("mpl"))

a,N,Q

(22, 3036893, 22)

In [48]:
#3127/59, 
#167659/431

In [54]:
from fractions import Fraction
from math import gcd,ceil,log2,pi
import time
start_time = time.time()

#a = 8
#N = 15

FACTOR_FOUND = False
ATTEMPT = 0
while not FACTOR_FOUND:
    ATTEMPT += 1
    print(f"\nAttempt {ATTEMPT}")

    phase = phase_estimation_2(
        c_amod15_2(a,N),
        psi_prep,
        precision=Q
    )
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator
    print(f"r:{r}")
    if phase != 0:
        # Guess for a factor is gcd(x^{r/2} - 1 , 15)
        guess = gcd(a ** (r // 2) - 1, N)
        if guess not in [1, N] and (N % guess) == 0:
            # Guess is a factor!
            print(f"Non-trivial factor found: {guess}")
            FACTOR_FOUND = True


end_time = time.time()  # End time
elapsed_time = end_time - start_time  # Calculate elapsed time

print(f"Elapsed time: {elapsed_time:.2f} seconds")


Attempt 1
num_qubits:22
gate_counts:112


C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_15416\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(


r:2097152

Attempt 2
num_qubits:22
gate_counts:112


C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_15416\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(


r:8192

Attempt 3
num_qubits:22
gate_counts:112


...


Attempt 18
num_qubits:22
gate_counts:112


C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_15416\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(


r:1100345

Attempt 19
num_qubits:22
gate_counts:112


C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_15416\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(


KeyboardInterrupt: 

In [ ]:
# theta = 0.025 # for 20 bits
"""
Attempt 8
num_qubits:20

gate_counts:102
C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_15416\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(
r:645341
Non-trivial factor found: 821
Elapsed time: 15.93 seconds

"""

In [ ]:
# theta = 0.01

"""
Attempt 26
num_qubits:18
gate_counts:92
C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_17896\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(
r:89183
Non-trivial factor found: 431
Elapsed time: 14.77 seconds

"""

In [ ]:
# theta = 0.1
"""
Attempt 15
num_qubits:18
gate_counts:92
C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_11796\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(
r:152134
Non-trivial factor found: 431
Elapsed time: 7.81 seconds


Attempt 18
num_qubits:18
gate_counts:92
C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_15416\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(
r:166153
Non-trivial factor found: 431
Elapsed time: 9.52 seconds

"""

In [ ]:
"""
Attempt 25
num_qubits:16
gate_counts:82
C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_11796\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(
C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_11796\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(
r:35446
Non-trivial factor found: 223
Elapsed time: 6.06 seconds


Attempt 12
num_qubits:16
C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_15416\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(
gate_counts:82
r:35669
Non-trivial factor found: 223
Elapsed time: 2.90 seconds
"""

In [ ]:
"""
Attempt 6
num_qubits:14
gate_counts:72
C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_11796\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(
C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_11796\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(
r:9980
Non-trivial factor found: 101
Elapsed time: 0.97 seconds


Attempt 1
num_qubits:14
gate_counts:72
r:9980
Non-trivial factor found: 101
Elapsed time: 0.13 seconds

"""

In [ ]:
"""
Attempt 96
num_qubits:12
gate_counts:62
C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_15416\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(
C:\Users\user.DESKTOP-JAJ50S0\AppData\Local\Temp\ipykernel_15416\3410879387.py:15: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  phase = phase_estimation_2(
r:2913
Non-trivial factor found: 53
Elapsed time: 14.00 seconds

"""

In [ ]:
"""
Attempt 53
num_qubits:10
gate_counts:52
r:561
Non-trivial factor found: 29
Elapsed time: 5.03 seconds

"""

In [ ]:
#clean up
#del U, Q
#del result, phase
#del psi_prep

A plot of the execution results is shown above. Compare this with the calculated values.

Footnotes

[1] This section is based on [10], [14], and [16].

[2] https://research.ibm.com/blog/factor-15-shors-algorithm

[3] https://en.wikipedia.org/wiki/Integer_factorization_records#Records_for_efforts_by_quantum_computers